In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# # Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1- Load the data
file_path = '/content/drive/MyDrive/Airline_Reviews.csv'
df = pd.read_csv(file_path)

In [ ]:
# show the dataset
df.head()

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,NaN,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,E120,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Embraer E120,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no


In [ ]:
# Inspect the data
print("Shape of the DataFrame----:", df.shape)
print(df.head())


Shape of the DataFrame----: (23171, 20)
   Unnamed: 0   Airline Name Overall_Rating  \
0           0    AB Aviation              9   
1           1    AB Aviation              1   
2           2    AB Aviation              1   
3           3  Adria Airways              1   
4           4  Adria Airways              1   

                            Review_Title          Review Date  Verified  \
0                "pretty decent airline"   11th November 2019      True   
1                   "Not a good airline"       25th June 2019      True   
2         "flight was fortunately short"       25th June 2019      True   
3    "I will never fly again with Adria"  28th September 2019     False   
4  "it ruined our last days of holidays"  24th September 2019      True   

                                              Review       Aircraft  \
0    Moroni to Moheli. Turned out to be a pretty ...            NaN   
1   Moroni to Anjouan. It is a very small airline...           E120   
2    Anjouan 

In [ ]:
# show more info the dataset
print("Info of the Data---------------- :")
print(df.info())

Info of the Data---------------- :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23171 entries, 0 to 23170
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              23171 non-null  int64  
 1   Airline Name            23171 non-null  object 
 2   Overall_Rating          23171 non-null  object 
 3   Review_Title            23171 non-null  object 
 4   Review Date             23171 non-null  object 
 5   Verified                23171 non-null  bool   
 6   Review                  23171 non-null  object 
 7   Aircraft                7129 non-null   object 
 8   Type Of Traveller       19433 non-null  object 
 9   Seat Type               22075 non-null  object 
 10  Route                   19343 non-null  object 
 11  Date Flown              19417 non-null  object 
 12  Seat Comfort            19016 non-null  float64
 13  Cabin Staff Service     18911 non-null  float64
 14  Foo

In [ ]:
print("Summary statistics of the Data----------------:")
print(df.describe())

Summary statistics of the Data----------------:
        Unnamed: 0  Seat Comfort  Cabin Staff Service  Food & Beverages  \
count  23171.00000  19016.000000         18911.000000      14500.000000   
mean   11585.00000      2.618321             2.871609          2.553586   
std     6689.03588      1.464844             1.604631          1.526314   
min        0.00000      0.000000             0.000000          0.000000   
25%     5792.50000      1.000000             1.000000          1.000000   
50%    11585.00000      3.000000             3.000000          2.000000   
75%    17377.50000      4.000000             4.000000          4.000000   
max    23170.00000      5.000000             5.000000          5.000000   

       Ground Service  Inflight Entertainment  Wifi & Connectivity  \
count    18378.000000            10829.000000          5920.000000   
mean         2.353738                2.178964             1.780405   
std          1.595747                1.488758             1.318800

In [ ]:
# change name colum
df.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

In [ ]:
# chake the null value
df.isnull().sum()

ID                            0
Airline Name                  0
Overall_Rating                0
Review_Title                  0
Review Date                   0
Verified                      0
Review                        0
Aircraft                  16042
Type Of Traveller          3738
Seat Type                  1096
Route                      3828
Date Flown                 3754
Seat Comfort               4155
Cabin Staff Service        4260
Food & Beverages           8671
Ground Service             4793
Inflight Entertainment    12342
Wifi & Connectivity       17251
Value For Money            1066
Recommended                   0
dtype: int64

In [ ]:
# drop colum Aircraft
df.drop(['Aircraft'],axis=1 , inplace= True)

In [ ]:
# chake the null value
df.isnull().sum()

ID                            0
Airline Name                  0
Overall_Rating                0
Review_Title                  0
Review Date                   0
Verified                      0
Review                        0
Type Of Traveller          3738
Seat Type                  1096
Route                      3828
Date Flown                 3754
Seat Comfort               4155
Cabin Staff Service        4260
Food & Beverages           8671
Ground Service             4793
Inflight Entertainment    12342
Wifi & Connectivity       17251
Value For Money            1066
Recommended                   0
dtype: int64

In [ ]:
# حذف الصفوف التي تحتوي على قيم فارغة في الأعمدة المحددة فقط
columns_to_check = ['Type Of Traveller','Seat Type','Route','Date Flown']
df= df.dropna(subset=columns_to_check)

In [ ]:
# chake the null value
df.isnull().sum()

ID                            0
Airline Name                  0
Overall_Rating                0
Review_Title                  0
Review Date                   0
Verified                      0
Review                        0
Type Of Traveller             0
Seat Type                     0
Route                         0
Date Flown                    0
Seat Comfort               1769
Cabin Staff Service        1874
Food & Beverages           6262
Ground Service             1023
Inflight Entertainment     9893
Wifi & Connectivity       13438
Value For Money               1
Recommended                   0
dtype: int64

In [ ]:
# Fill missing values:  Seat Comfort  , Cabin Staff Service ,Inflight Entertainment,Wifi & Connectivity,Food & Beverages, Ground Service  , Value For Money


df['Seat Comfort'] = df['Seat Comfort'].fillna(4).replace([np.inf, -np.inf], 999).astype(int)
df['Cabin Staff Service'] = df['Cabin Staff Service'].fillna(4).replace([np.inf, -np.inf], 999).astype(int)
df['Inflight Entertainment'] = df['Inflight Entertainment'].fillna(3).replace([np.inf, -np.inf], 999).astype(int)
df['Wifi & Connectivity'] = df['Wifi & Connectivity'].fillna(3).replace([np.inf, -np.inf], 999).astype(int)
df['Food & Beverages'] = df['Food & Beverages'].fillna(3).replace([np.inf, -np.inf], 999).astype(int)
df['Ground Service'] = df['Ground Service'].fillna(3).replace([np.inf, -np.inf], 999).astype(int)
df['Value For Money'] = df['Value For Money'].fillna(4).replace([np.inf, -np.inf], 999).astype(int)


<ipython-input-197-a6a1f08b7fcf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Seat Comfort'] = df['Seat Comfort'].fillna(4).replace([np.inf, -np.inf], 999).astype(int)
<ipython-input-197-a6a1f08b7fcf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cabin Staff Service'] = df['Cabin Staff Service'].fillna(4).replace([np.inf, -np.inf], 999).astype(int)
<ipython-input-197-a6a1f08b7fcf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [ ]:
# Convert values ​​in 'Recommended' column from 'yes' and 'no' to 'True' and 'False'
df['Recommended'] = df['Recommended'].apply(lambda x: True if x == 'yes' else False)


<ipython-input-198-202d773c1a45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Recommended'] = df['Recommended'].apply(lambda x: True if x == 'yes' else False)


In [ ]:
# chake the null value
df.isnull().sum()

ID                        0
Airline Name              0
Overall_Rating            0
Review_Title              0
Review Date               0
Verified                  0
Review                    0
Type Of Traveller         0
Seat Type                 0
Route                     0
Date Flown                0
Seat Comfort              0
Cabin Staff Service       0
Food & Beverages          0
Ground Service            0
Inflight Entertainment    0
Wifi & Connectivity       0
Value For Money           0
Recommended               0
dtype: int64

In [ ]:
# Function to remove 'to' and any word that comes after it
def remove_after_to(route):
    if pd.isna(route):
        return route
    if 'to' in route:
        parts = route.split(' to ')
        return parts[0]
    return route

# Apply the function to the Route column and create a new column
df['Departure city'] = df['Route'].apply(remove_after_to)

df.head()



<ipython-input-200-4e0e16acc807>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Departure city'] = df['Route'].apply(remove_after_to)


,ID,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended,Departure city
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,Solo Leisure,Economy Class,Moroni to Moheli,November 2019,4,5,4,4,3,3,3,True,Moroni
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,Solo Leisure,Economy Class,Moroni to Anjouan,June 2019,2,2,1,1,3,3,2,False,Moroni
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,Solo Leisure,Economy Class,Anjouan to Dzaoudzi,June 2019,2,1,1,1,3,3,2,False,Anjouan
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,Solo Leisure,Economy Class,Frankfurt to Pristina,September 2019,1,1,3,1,3,3,1,False,Frankfurt
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,September 2019,1,1,1,1,1,1,1,False,Sofia


In [ ]:

def extract_destination(route):
    if pd.isna(route):  # Handle missing values
        return route
    if 'to' in route:
        parts = route.split(' to ')
        if len(parts) > 1:  # Check if there's a destination
            destination = parts[1]
            if ' via ' in destination:
                return destination.split(' via ')[0]
            return destination
    return route  # Return the original route if no destination is found

# Apply the function to the Route column and create a new column
df['Arrival city'] = df['Route'].apply(extract_destination)

In [ ]:
# drop colum Route
df.drop(['Route'],axis=1 , inplace= True)

In [ ]:
# add colum year from Review Date and change type for Review Date to string
try:
    df['Review Date'] = pd.to_datetime(df['Review Date'], format='%dth %B %Y')
except ValueError:
    df['Review Date'] = pd.to_datetime(df['Review Date'], errors='coerce')
df['Year'] = df['Review Date'].dt.year

df['Review Date'] = df['Review Date'].astype(str)



In [ ]:
# change type for Review Date to integer
df['Year'] = df['Year'].fillna(0).replace([np.inf, -np.inf], 0).astype(int)


In [ ]:
# change type from object to date
df['Date Flown'] = pd.to_datetime(df['Date Flown'], format='%B %Y')

df['Month Flown'] = df['Date Flown'].dt.month_name()

In [ ]:
# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize text
    words = word_tokenize(text)
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Join words back to a single string
    cleaned_text = ' '.join(words)
    return cleaned_text



In [ ]:
# Apply the clean_text function to the Review_Title and Review columns
df['Cleaned_Review_Title'] = df['Review_Title'].apply(clean_text)
df['Cleaned_Review'] = df['Review'].apply(clean_text)

In [ ]:
# Save the cleaned data to a new CSV file
output_path = '/content/Cleaned_Airline_Reviews.csv'
df.to_csv(output_path, index=False)